In [ ]:
!pip install pyspark

     |████████████████████████████████| 212.4 MB 61 kB/s 
     |████████████████████████████████| 198 kB 51.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=a769357cb7f7721324211e1df184c463f920b755a85632f025ea28d49f3226dd
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark


**Merging Datasets**

In [ ]:
#import statements
from pyspark.sql import SparkSession

spark = SparkSession.builder.enableHiveSupport().appName('ReadWriteData').getOrCreate()
sc = spark.sparkContext

In [ ]:
from pyspark.sql.types import FloatType
title_basics = spark.read.csv("drive/MyDrive/ADM/title.basics.tsv", sep=r'\t', inferSchema=True, header=True)
title_basics = title_basics.withColumn("startYear",title_basics["startYear"].cast(FloatType()))
title_basics.filter(title_basics.startYear <= 2005.0)

DataFrame[tconst: string, titleType: string, primaryTitle: string, originalTitle: string, isAdult: string, startYear: float, endYear: string, runtimeMinutes: string, genres: string]

In [ ]:
title_crew = spark.read.csv("drive/MyDrive/ADM/title.crew.tsv", sep=r'\t', inferSchema=True, header=True)

In [ ]:
master_data = title_basics.join(title_crew, on = 'tconst')

In [ ]:
title_principals = spark.read.csv("drive/MyDrive/ADM/title.principals.tsv", sep=r'\t', inferSchema=True, header=True)
name_basics = spark.read.csv("drive/MyDrive/ADM/name.basics.tsv", sep=r'\t', inferSchema=True, header=True)
title_principals = title_principals.join(name_basics, on = 'nconst')
cols = ['tconst', 'category', 'primaryName']
title_principals.select(*cols)

DataFrame[tconst: string, category: string, primaryName: string]

In [ ]:
master_data = master_data.join(title_principals, on = 'tconst')

In [ ]:
title_ratings = spark.read.csv("drive/MyDrive/ADM/title.ratings.tsv", sep=r'\t', inferSchema=True, header=True)

In [ ]:
master_data = master_data.join(title_ratings, on = 'tconst')

In [ ]:
master_data.write.save('drive/MyDrive/ADM/masterdata.csv', format='csv')

In [ ]:
netflix_data = spark.read.csv('drive/MyDrive/ADM/movie_c.csv', inferSchema=True, header=False)

In [ ]:
# _c0 = movie id
# _c1 = customer id
# _c2 = rating
# _c3 = year
# _c4 = movie name
netflix_data.show(1)

+----+-----+---+----+--------------------+
| _c0|  _c1|_c2| _c3|                 _c4|
+----+-----+---+----+--------------------+
|4265|91502|3.0|2002|National Geograph...|
+----+-----+---+----+--------------------+
only showing top 1 row



In [ ]:
netflix_data = netflix_data.join(master_data, (netflix_data._c4 == master_data.primaryTitle) & (netflix_data._c3 == master_data.startYear))

In [ ]:
netflix_data.write.csv('drive/MyDrive/ADM/netflixdata.csv', header=True)

In [ ]:
netflix_data.show(50)

+---+-------+---+----+--------------------+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+---------------+---------+---------+---------+--------+---------------+--------------------+---------------+--------------+---------+---------+--------------------+--------------------+-------------+--------+
|_c0|    _c1|_c2| _c3|                 _c4|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|         genres|directors|  writers|   nconst|ordering|       category|                 job|     characters|   primaryName|birthYear|deathYear|   primaryProfession|      knownForTitles|averageRating|numVotes|
+---+-------+---+----+--------------------+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+---------------+---------+---------+---------+--------+---------------+--------------------+---------------+--------------+---------+---------+-

In [ ]:
netflix_data.toPandas().to_csv('drive/MyDrive/ADM/netflixdata_pandas.csv', header=True, index=False)

Py4JJavaError: ignored